<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import gensim
import numpy as np
import pandas as pd

import nltk
import time
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud, STOPWORDS
from gensim.parsing.preprocessing import stem_text
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from gensim import corpora, models, similarities
from gensim.models import Word2Vec, TfidfModel, LsiModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorboard as tb
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.python.framework import ops
#tf.gfile = tf.io.gfile
#tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import random
import pathlib
import matplotlib.pyplot as plt
AUTOTUNE = tf.data.experimental.AUTOTUNE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
#!pip install git+https://github.com/huggingface/transformers
!pip install glove_python
from glove import Corpus, Glove# creating a corpus object

     |████████████████████████████████| 266kB 2.6MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700293 sha256=0a71a8958a6feae378382d5c206172ee0a5e40a9d5013cba082efd7b8e5d131f
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [3]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        pickle_object = pickle.load(input)
    return  pickle_object

In [4]:
!mkdir imdb

def pre_process(data):
  _sent=[]
  for i in data:
    sent = word_tokenize(i)
    sent = [w.lower() for w in sent]
    #sent = [word for word in sent if word.isalpha()]
    sent = [c for c in sent if not c.isdigit()]
    sent = [stem_text(w) for w in sent]
    #sent = [wordnet_lemmatizer.lemmatize(w) for w in sent]
    #sent = [word for word in sent if word not in stoplist]
    _sent.append(sent)
  return _sent

from keras.datasets import imdb
data = imdb.load_data(path='/content/imdb/data',
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3)
save_object(data, '/content/imdb/data')
word_to_idx_dict = imdb.get_word_index(path="/content/imdb/imdb_word_index.json")
n = len(word_to_idx_dict)
print("Number distinct words in reviews = %d \n" % n)

words_freq_list = []
for (k,v) in word_to_idx_dict.items():
  words_freq_list.append((k,v))

sorted_list = sorted(words_freq_list, key=lambda x: x[1])

print("Ten most common words: \n")
print(sorted_list[0:10])

print("\nLast five least common words: \n")
print(sorted_list[-5:])

Using TensorFlow backend.


1646592/1641221 [==============================] - 1s 1us/step
Number distinct words in reviews = 88584 

Ten most common words: 

[('the', 1), ('and', 2), ('a', 3), ('of', 4), ('to', 5), ('is', 6), ('br', 7), ('in', 8), ('it', 9), ('i', 10)]

Last five least common words: 

[("pipe's", 88580), ('copywrite', 88581), ('artbox', 88582), ("voorhees'", 88583), ("'l'", 88584)]


In [5]:
#d = load_object('/content/imdb/data')
#d

In [6]:
def data_module(reviews, name, location, output_location, dict_location, max_seq_length=100):
  if reviews==False:
    dataset1 = pd.read_csv(location)
    dataset = dataset1.iloc[0:1000,:]
  else:
    dataset = load_object(location)
  if name == 'word2vec':
    data_corpus, users_total = data_word2vec(reviews,dataset,max_seq_length)
  elif name == 'doc2vec':
    data_corpus, users_total = data_doc2vec(reviews,dataset,max_seq_length)
  elif name == 'lsi':
    data_corpus, users_total = data_lsi(reviews,dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'tfidf':
    data_corpus, users_total = data_lsi(reviews,dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'glove':
    data_corpus, users_total = data_word2vec(reviews,dataset,max_seq_length)
  elif name == 'hashing':
    data_corpus, users_total = data_hashing(reviews,dataset,max_seq_length)
  elif name == 'cooccur':
    data_corpus, users_total = data_hashing(reviews,dataset,max_seq_length)
  #elif name == 'fasttext':
   # data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  #data_corpus.save('/content/drive/My Drive/Movielensdata/ml25m/data_corpus')
  #print(data_corpus)
  save_object(obj=data_corpus, filename=output_location)
  #return data_corpus, users_total

def data_word2vec(review,dataset,max_seq_length):
  if review==False:
    dataset.sort_values(by=['userId','timestamp'],inplace=True)
    user_total = len(dataset['userId'].unique())
    
    #Selecting the most recent movies rated by each user and padding if necessary
    movie_list = []
    for i in range(user_total):
      list1 = []
      list1 = dataset.loc[dataset['userId'] ==(i+1),['movieId']]['movieId'].tolist()
      if len(list1)>max_seq_length:
        list1 = list1[(len(list1)-max_seq_length):]
      elif len(list1)<max_seq_length:
        list1 = list1+[0 for j in range((max_seq_length-len(list1)))]
        #for j in range((max_seq_length-len(list1))):
        # list1.append(0)
      movie_list.append(list1)

    #Selecting the most recent ratings rated by each user and padding if necessary
    rating_list =[]
    for i in range(user_total):
      list2 = []
      list2 = dataset.loc[dataset['userId'] ==(i+1),['rating']]['rating'].tolist()
      if len(list2)>max_seq_length:
        list2 = list2[(len(list2)-max_seq_length):]
      elif len(list2)<max_seq_length:
        list2 = list2+[0 for j in range((max_seq_length-len(list2)))]
        #for j in range((max_seq_length-len(list2))):
        # list2.append(0)
      rating_list.append(list2)

    #Creating user_id level transpose matrices
    movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(user_total)])
    movies_transpose.index.names = ['userId']
    #print(movies_transpose)

    ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(user_total)])
    ratings_transpose.index.names = ['userId']
    #print(ratings_transpose)

    # Select features from original dataset to form a new dataframe 
    df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
    df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
    df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
    sent = [row.split(',') for row in df_clean['clean']]

    return sent, user_total
  else:
    X_train = dataset[0][0]
    Y_train = dataset[0][1]
    X_test, X_val, Y_test, Y_val = train_test_split(data[1][0], data[1][1], test_size=0.4, shuffle=False)
    
    reverse_word_index = dict([(value, key) for (key, value) in word_to_idx_dict.items()])
    train_reviews=[]
    for j in X_train:
      train_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    test_reviews=[]
    for j in X_test:
      test_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    val_reviews=[]
    for j in X_val:
      val_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    train_sent=pre_process(train_reviews)
    save_object(train_sent,'/content/imdb/train_sent')
    test_sent=pre_process(test_reviews)
    save_object(test_sent,'/content/imdb/test_sent')
    val_sent=pre_process(val_reviews)
    save_object(val_sent,'/content/imdb/val_sent')
    save_object(Y_train, '/content/imdb/Y_train')
    save_object(Y_test, '/content/imdb/Y_test')
    save_object(Y_val, '/content/imdb/Y_val')

    return train_sent, len(train_sent)

def data_doc2vec(review,dataset,max_seq_length):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  tagged_data = []
  tags = []
  
  for i in range(user_total):
    tagged_data = tagged_data + [TaggedDocument(words=Sent[i], tags=[str(i)])]

  return tagged_data, user_total

def data_lsi(review,dataset,max_seq_length,dict_loc):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  dictionary = corpora.Dictionary(Sent)
  #print(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in Sent]
  dictionary.save(dict_loc)
  #corpus = np.array([[(id, freq) for id, freq in cp] for cp in corp])
  #corpus = gensim.matutils.Dense2Corpus(np.array(Sent),documents_columns=False)

  return corpus, user_total

def data_hashing(review,dataset,max_seq_length):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  corpus = [str(str(doc)[1:-1]) for doc in Sent]
  return corpus, user_total

In [7]:
def embedding_model(name,Data_location,model_save_location,matrix_location, maxi_features, vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 100,alpha = 0.025,min_alpha=0.00025,dm=1):
  Data = load_object(Data_location)
  #print(Data)
  if name == 'word2vec':
    word2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers)
    #voc = model1.wv
    #words = list(model1.wv.vocab)
    #vectors = model1[model1.wv.vocab]
  elif name == 'doc2vec':
    doc2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, alpha_=alpha, size_of_window=size_window, no_workers=num_workers, max_epochs=max_num_epochs,min_alpha_=min_alpha, minimum_count=mini_count, dms=dm)
  elif name == 'lsi':
    lsi(input_data=Data, save_loc=model_save_location, total_topics=topics)
  elif name == 'tfidf':
    tfidf(input_data=Data,save_loc=model_save_location)
  elif name == 'glove':
    glove_model(input_data=Data, vec_dims=vector_dims, size_of_window=size_window, save_loc=model_save_location, num_epochs=max_num_epochs, alpha_=0.05, num_threads=4)
  elif name == 'hashing':
    hashing(input_data=Data, vec_dims=vector_dims, save_loc=model_save_location)
  elif name == 'cooccur':
    co_occur(input_data=Data, maximum_features=maxi_features, save_loc=model_save_location, matrix_loc = matrix_location)
  #elif name == 'fasttext':
   # fast_text(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers, alpha_=0.025)

def word2vec(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers):
  model = Word2Vec(input_data,min_count=minimum_count,size= vec_dims,workers=no_workers, window =size_of_window, sg = SG)
  model.save(save_loc)

def doc2vec(input_data,save_loc,vec_dims,alpha_,size_of_window,min_alpha_,minimum_count,dms,no_workers,max_epochs):
  model = Doc2Vec(size=vec_dims,
                alpha=alpha_, 
                min_alpha=min_alpha_,
                window = size_of_window,
                min_count=minimum_count,
                dm =dms)
  model.build_vocab(input_data)

  for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(input_data, total_examples=model.corpus_count, epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
  model.save(save_loc)

def lsi(input_data,save_loc,total_topics):
  model = models.LsiModel(corpus=input_data, num_topics=total_topics)
  index = similarities.MatrixSimilarity(model[input_data])
  lsi_data = model[input_data]
  lsi_topics = model.print_topics()
  #for topic in lsi_topics:
    #print(topic)
  model.save(save_loc)

def tfidf(input_data,save_loc):
  model = models.TfidfModel(corpus=input_data, normalize=True)
  tfidf_data = model[input_data]
  model.save(save_loc)

def glove_model(input_data,vec_dims,size_of_window,save_loc,alpha_=0.05,num_epochs=30, num_threads=4):
  #importing the glove library
  corpus = Corpus() #training the corpus to generate the co occurence matrix which is used in GloVe
  corpus.fit(input_data, window=size_of_window)#creating a Glove object which will use the matrix created in the above lines to create embeddings
  #We can set the learning rate as it uses Gradient Descent and number of components
  glove = Glove(no_components=vec_dims, learning_rate=alpha_) 
  glove.fit(corpus.matrix, epochs=num_epochs, no_threads=4, verbose=True)
  glove.add_dictionary(corpus.dictionary)
  glove.save(save_loc)

def fast_text(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers,alpha_=0.025):
  model = FastText(min_count=minimum_count, alpha=alpha_, size= vec_dims, workers=no_workers, window =size_of_window)
  model.build_vocab(input_data)
  model.train(input_data, epochs=model.epochs, total_examples=model.corpus_count, total_words=model.corpus_total_words)
  model.save(save_loc)

def hashing(input_data,vec_dims,save_loc):
  model = HashingVectorizer(n_features=vec_dims)
  model.transform(input_data)
  #vectors = model.toarray()
  #vocab = model.get_feature_names()
  save_object(obj=model, filename=save_loc)

def co_occur(input_data, save_loc, matrix_loc, maximum_features):
  model = CountVectorizer(ngram_range=(1,1),max_features=maximum_features, token_pattern= r"(?u)\b\w+\b")
  X = model.fit_transform(input_data)
  Xc = (X.T * X)
  Xc.setdiag(0)
  #cooccur = Xc.todense()
  names = model.get_feature_names() # This are the entity names (i.e. keywords)
  save_object(obj=names, filename='/content/drive/My Drive/Movielensdata/ml25m/cooccur/vocab')
  df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
  save_object(obj=model, filename=save_loc)
  save_object(obj=df, filename=matrix_loc)


In [8]:
def scoring_module(name,model_loc,data_loc,dict_loc,matrix_location):
  data = load_object(data_loc)
  if name == 'word2vec':
    scores_list = score_word2vec(model_location=model_loc,input_data=data)
  elif name == 'doc2vec':
    scores_list = score_doc2vec(model_location=model_loc,input_data=data)
  elif name == 'lsi':
    scores_list = score_lsi(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'tfidf':
    scores_list = score_tfidf(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'glove':
    scores_list = score_glove(model_location=model_loc, input_data=data)
  elif name == 'hashing':
    scores_list = score_hashing(model_location=model_loc, input_data=data)
  elif name == 'cooccur':
    scores_list = score_cooccur(matrix_loc=matrix_location, input_data=data)
  #elif name == 'fasttext':
   # scores_list = score_fasttext(model_location=model_loc, input_data=data)
  
  return scores_list

def score_word2vec(model_location,input_data):
  model = Word2Vec.load(model_location)
  scored_data = [model.wv[text] for text in input_data]
  #print(model.wv['please'])
  return scored_data

def score_doc2vec(model_location,input_data):
  model = Doc2Vec.load(model_location)
  scored_data = [model.infer_vector(text) for text in input_data]
  return scored_data

def score_glove(model_location,input_data):
  model = Glove.load(model_location)
  scored_data = []
  #doc = input_data[0]
  for doc in input_data:
    list1 = []
    for text in doc:
      list1.append(model.word_vectors[model.dictionary[text]])
    scored_data.append(list1)
  #scored_data = [model.word_vectors[model.dictionary[text]] for doc in input_data for text in doc]
  return scored_data

def score_fasttext(model_location, input_data):
  model = FastText(model_location)
  scored_data = [model[text] for text in input_data]
  return scored_data

def score_lsi(model_location, input_data, dict_location):
  model = LsiModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  scored_dat = np.zeros((len(scored_data),len(scored_data[0])))
  for i in range(len(scored_data)):
    for k in range(len(scored_data[i])):
      scored_dat[i][k] = scored_data[i][k][1]
  return scored_dat

def score_tfidf(model_location, input_data, dict_location):
  model = TfidfModel.load(model_location)
  dictionary = corpora.Dictionary.load(dict_location)
  #print(dictionary)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  scored_data_ = gensim.matutils.corpus2dense(scored_data, num_terms=len(dictionary), num_docs=len(scored_data))
  #scored_data_ = gensim.matutils.corpus2csc(scored_data, num_terms=len(dictionary))
  #scored_data_= np.zeros((len(scored_data), len(dictionary)), dtype=np.float64)
  #for i in range(len(scored_data)):
    #print(scored_data[i][k][0])
   # for k in range(len(scored_data[i])):
    #  scored_data_[i][scored_data[i][k][0]] = scored_data[i][k][1]
  #scored_data = np.array(scored_data)
  #scored_data = scored_data.transpose()
  #scored_data_ = gensim.matutils.corpus2dense(scored_data, num_terms=len(dictionary), num_docs=len(scored_data))
  #for i in range(len(scored_data)):
   # scored_data_.append(scored_data[i].todense())
  #tfidf_token= np.zeros((len(scored_data), 50), dtype=np.float64)
  #tfidf_vals= np.zeros((len(scored_data), 50), dtype=np.float64)
  #for i in range(len(input_data)):
   # for k in range(len(scored_data[i])):
    #  print(tfidf_vals[i][k])
      #tfidf_token[i][k]=(list(scored_data))[i][k][0]
      #tfidf_vals[i][k]=scored_data[i][k][1]
  #tfidf_list=scored_data
  #print(list(tfidf_data))
  #print(tfidf_token)
  #print(tfidf_vals)
  return scored_data_, scored_data

def score_hashing(model_location, input_data):
  corpus = [str(str(doc)[1:-1]) for doc in input_data]
  model = load_object(model_location)
  X = model.transform(corpus)
  Y = X.toarray()
  #Doc_Term_Matrix = pd.DataFrame(X.toarray())
  #Doc_Term_Matrix
  return Y

def score_cooccur(matrix_loc, input_data):
  cooccur_matrix = load_object(matrix_loc)
  #print(cooccur_matrix)
  scored_data = [cooccur_matrix.loc[text].tolist() for doc in input_data for text in doc]
  return scored_data

In [9]:
def sent_embedding(name,score_list):
  if (name=='word2vec'):
    sent_list = []
    for i in range(len(score_list)):
      sum = np.zeros((10))
      for j in range(len(score_list[i])):
        sum = sum + score_list[i][j]
      sum = sum/(len(score_list[i]))
      sent_list.append(sum)
    sent_tensor = tf.convert_to_tensor(sent_list, dtype=tf.float32)
  elif name=='glove':
    sent_list = []
    for i in range(len(score_list)):
      sum = np.zeros((10))
      for j in range(len(score_list[i])):
        sum = sum+score_list[i][j]
      sum = sum/(len(score_list[i]))
      sent_list.append(sum)
    sent_tensor = tf.convert_to_tensor(sent_list, dtype=tf.float32)
  elif name=='doc2vec':
    sent_tensor = tf.convert_to_tensor(score_list, dtype=tf.float32)
  elif name=='tfidf':
    sent_tensor = tf.convert_to_tensor(score_list, dtype=tf.float32)
  elif name=='lsi':
    sent_tensor = tf.convert_to_tensor(score_list, dtype=tf.float32)
  elif name=='hashing':
    sent_tensor = tf.convert_to_tensor(score_list, dtype=tf.float32)
  return sent_tensor

def Model_build(X,Y,X_predict,Y_predict,save_location,batches_size=100):
  ops.reset_default_graph()
  Model = Sequential()
  #Model.add(Dense(20, activation='relu'))
  #Model.add(Dropout(0.3))
  Model.add(Dense(5, activation='relu'))
  #Model.add(Dropout(0.3))
  Model.add(Dense(1, activation='sigmoid'))
  #Model.add(Dense(1))
  #Model.add(Activation('softmax'))

  opt = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

  Model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  #Model.summary()

  steps_test = int(len(X)/batches_size)
  steps_val = int(len(X_predict)/batches_size)

  Model.fit(X, Y, epochs=10, batch_size=batches_size)

  #print('The train accuracies are as given below :')
  #print(History.history['accuracy'])
  Model.save('/content/imdb/model')
  #print(Model.metrics_names)
  sentiment_score = Model.predict(X_predict, batch_size=batches_size, steps=steps_val, workers=3)
  metrics = Model.evaluate(x=X_predict, y=Y_predict, batch_size=batches_size, verbose=1, steps=steps_val, max_queue_size=10, return_dict=False)
  accuracy = metrics[1]
  loss = metrics[0]
  save_object(sentiment_score,save_location)
  return sentiment_score, accuracy

1) Name can be 'word2vec' or 'doc2vec' or 'lsi' or 'tfidf' or 'glove' or 'cooccur'.

2) location is the path to input .csv file

3) output_location is the path to save the processed data

4) dict_location is the path to dictionary of LSI or TF-IDF

Change name, output_location for all models and dict_location for LSI and TF-IDF

In [10]:
#data_module(reviews=False,name='glove',max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/ratings.csv',output_location='/content/drive/My Drive/Movielensdata/ml25m/glove/data', dict_location='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict')

Change name, Data_location and model_save_location for all models

In [11]:
#embedding_model(name='glove',Data_location='/content/drive/My Drive/Movielensdata/ml25m/glove/data',model_save_location='/content/drive/My Drive/Movielensdata/ml25m/glove/glv', matrix_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix', vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1,maxi_features=None)

In [12]:
#scoring_input = [['899', '2161', '3949', '5878', '1175', '1237', '8154', '2843', '7365', '4422', '6016'],['1080', '3114', '3671', '2791', '1288', '1', '541', '2692', '7323', '8014', '6370', '4703', '5147']]
#save_object(scoring_input,'/content/drive/My Drive/Movielensdata/ml25m/scoring_input')

Change name, model_loc or all models and dict_loc for LSI and TF-IDF 

In [13]:
#scored_list = scoring_module(name='glove', model_loc='/content/drive/My Drive/Movielensdata/ml25m/glove/glv', data_loc='/content/drive/My Drive/Movielensdata/ml25m/scoring_input', dict_loc='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix' )

In [14]:
#print(scored_list)

In [15]:
data = load_object('/content/imdb/data')

In [16]:
data_module(reviews=True, name='word2vec',max_seq_length=100,location='/content/imdb/data',output_location='/content/imdb/train_sent', dict_location='/content/imdb/tfidf_dict')

In [17]:
embedding_model(name='word2vec',Data_location='/content/imdb/train_sent',model_save_location='/content/imdb/trained_model', matrix_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix',maxi_features=25000, vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [18]:
scored_test= scoring_module(name='word2vec', model_loc='/content/imdb/trained_model', data_loc='/content/imdb/test_sent', dict_loc='/content/imdb/tfidf_dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix' )

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [19]:
scored_val = scoring_module(name='word2vec', model_loc='/content/imdb/trained_model', data_loc='/content/imdb/val_sent', dict_loc='/content/imdb/tfidf_dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix' )

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [20]:
#score[0]
#print(score[1])

In [21]:
#print(scored_val[0])
print(len(scored_val))

10000


In [22]:
#print(scored_test[0])

In [23]:
test_tensor = sent_embedding(name='word2vec',score_list=scored_test)
val_tensor = sent_embedding(name='word2vec',score_list=scored_val)

Y_test = load_object('/content/imdb/Y_test')
Y_val = load_object('/content/imdb/Y_val')

In [24]:
print(test_tensor.shape)
print(val_tensor.shape)

(15000, 10)
(10000, 10)


In [25]:
sentiment_scores, accuracy = Model_build(X=test_tensor, Y=Y_test, X_predict=val_tensor, Y_predict=Y_val,save_location='/content/imdb/sentiment_word2vec',batches_size=100)

Epoch 1/10
150/150 [==============================] - 0s 994us/step - loss: 0.6950 - accuracy: 0.5726
Epoch 2/10
150/150 [==============================] - 0s 1ms/step - loss: 0.6508 - accuracy: 0.6550
Epoch 3/10
150/150 [==============================] - 0s 1ms/step - loss: 0.6269 - accuracy: 0.6668
Epoch 4/10
150/150 [==============================] - 0s 1ms/step - loss: 0.6174 - accuracy: 0.6682
Epoch 5/10
150/150 [==============================] - 0s 1ms/step - loss: 0.6130 - accuracy: 0.6687
Epoch 6/10
150/150 [==============================] - 0s 994us/step - loss: 0.6123 - accuracy: 0.6711
Epoch 7/10
150/150 [==============================] - 0s 957us/step - loss: 0.6121 - accuracy: 0.6699
Epoch 8/10
150/150 [==============================] - 0s 951us/step - loss: 0.6106 - accuracy: 0.6705
Epoch 9/10
150/150 [==============================] - 0s 955us/step - loss: 0.6091 - accuracy: 0.6721
Epoch 10/10
150/150 [==============================] - 0s 972us/step - loss: 0.6102 - accu

In [26]:
sentiment_scores[0:10]

array([[0.7544033 ],
       [0.6944938 ],
       [0.20898059],
       [0.63125134],
       [0.25758255],
       [0.29301885],
       [0.1977123 ],
       [0.53893894],
       [0.31051928],
       [0.67285866]], dtype=float32)

In [27]:
Y_val[0:10]

array([1, 1, 1, 1, 0, 0, 0, 1, 0, 0])

In [28]:
accuracy

0.6686000227928162